In [1]:
import os
import sys
from glob import glob
import pandas as pd
from openai_client import client, call_openai_api
from load_configs import openai_api_key

# RunnablePassthrough

In [2]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}


def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] # Parsing logic
}

chain.invoke('hello') # {'original': 'completion', 'parsed': 'noitelpmoc'}

{'original': 'completion', 'parsed': 'noitelpmoc'}

In [3]:
def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

{'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

# oo

In [4]:
data_path = os.path.join("../data")
os.path.exists(data_path)
csv_files = glob(f"{data_path}/*.csv")
print(csv_files)

['../data/country_eng.csv', '../data/custom_2020.csv', '../data/Mobile-Price-Prediction-cleaned_data.csv']


In [5]:
sfile = csv_files[0]
df_small = pd.read_csv(sfile)
df_small.count()

Unnamed: 0      232
Country         232
Country_name    232
Area            232
dtype: int64

In [6]:
lfile = csv_files[1]
df_large = pd.read_csv(lfile)
df_large.count()

exp_imp    3299804
Year       3299804
month      3299804
ym         3299804
Country    3299804
Custom     3299804
hs2        3299804
hs4        3299804
hs6        3299804
hs9        3299804
Q1         3299804
Q2         3299804
Value      3299804
dtype: int64

In [7]:
from langchain_openai import ChatOpenAI, OpenAI
llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o"
    )

In [8]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent


In [9]:
sfile

'../data/country_eng.csv'

In [10]:
agent = create_csv_agent(
    llm,
    sfile,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    verbose=True
)
agent.run("how many rows are there?")

/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


232There are 232 rows in the dataframe `df`.

> Finished chain.


'There are 232 rows in the dataframe `df`.'

In [11]:
agent = create_csv_agent(
    llm,
    sfile,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    verbose=True
)
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To determine the number of rows in the dataframe `df`, I can use the `shape` attribute, which returns a tuple where the first element is the number of rows. 
Action: python_repl_ast
Action Input: df.shape(232, 4)I now know the final answer
Final Answer: There are 232 rows in the dataframe `df`.

> Finished chain.


'There are 232 rows in the dataframe `df`.'

In [12]:
agent = create_csv_agent(
    llm,
    sfile,
    agent_type="tool-calling", #AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    verbose=True,
    # return_intermediate_steps=True,
    # return_direct=True,
    # response_format='content_and_artifact'
)
res = agent.run(input="how many rows are there?", 
                # return_direct=True,
    # response_format='content_and_artifact'
                # return_intermediate_steps=True,
               )
print(res)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


232There are 232 rows in the dataframe.

> Finished chain.
There are 232 rows in the dataframe.


In [13]:
# agent.get_allowed_tools()
agent.OutputType

typing.Dict[str, typing.Any]

In [14]:
agent.input_keys

['input']

In [15]:
agent.output_keys

['output']

In [16]:
# agent.return_values

In [17]:
agent.tools[0].return_direct = True
agent.tools[0].response_format='content'#'content_and_artifact'

In [18]:
res = agent.run(input="how many rows are there?",)
print(res)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


232


> Finished chain.
232


In [19]:
res = agent.run(input="list all the countries",)
print(res)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Country_name'].tolist()"}`


['Korea', 'North_Korea', 'China', 'Taiwan', 'Mongolia', 'Hong_Kong', 'Viet_Nam', 'Thailand', 'Singapore', 'Malaysia', 'Brunei', 'Philippines', 'Indonesia', 'Cambodia', 'Lao', 'Myanmar', 'India', 'Pakistan', 'Sri_Lanka', 'Maldives', 'Bangladesh', 'Timor-Leste', 'Macao', 'Afghanistan', 'Nepal', 'Bhutan', 'Iran', 'Iraq', 'Bahrain', 'Saudi_Arabia', 'Kuwait', 'Qatar', 'Oman', 'Israel', 'Jordan', 'Syria', 'Lebanon', 'United_Arab_Emirates', 'Yemen', 'Azerbaijan', 'Armenia', 'Uzbekistan', 'Kazakhstan', 'Kyrgyz', 'Tajikistan', 'Turkmenistan', 'Georgia', 'West_Bank_and_Gaza_Strip', 'Iceland', 'Norway', 'Sweden', 'Denmark', 'United_Kingdom', 'Ireland', 'Netherlands', 'Belgium', 'Luxembourg', 'France', 'Monaco', 'Andorra', 'Germany', 'Switzerland', 'Azores_(Portugal)', 'Portugal', 'Spain', 'Gibraltar_(UK)', 'Italy', 'Malta', 'Finland', 'Poland', 'Russia', 'Austria', 'Hungary', 'Se

In [20]:
res = agent.run(input=" give me the 10 country name and the area both",)
print(res)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[['Country_name', 'Area']].head(10)"}`


  Country_name  Area
0        Korea  Asia
1  North_Korea  Asia
2        China  Asia
3       Taiwan  Asia
4     Mongolia  Asia
5    Hong_Kong  Asia
6     Viet_Nam  Asia
7     Thailand  Asia
8    Singapore  Asia
9     Malaysia  Asia


> Finished chain.
  Country_name  Area
0        Korea  Asia
1  North_Korea  Asia
2        China  Asia
3       Taiwan  Asia
4     Mongolia  Asia
5    Hong_Kong  Asia
6     Viet_Nam  Asia
7     Thailand  Asia
8    Singapore  Asia
9     Malaysia  Asia


In [21]:
res = agent.run(input=" give me the 10 country name first and then give me the 10 area",)
print(res)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Country_name'].head(10).tolist()"}`


['Korea', 'North_Korea', 'China', 'Taiwan', 'Mongolia', 'Hong_Kong', 'Viet_Nam', 'Thailand', 'Singapore', 'Malaysia']

Invoking: `python_repl_ast` with `{'query': "df['Area'].head(10).tolist()"}`


['Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia']
The first 10 country names are:
1. Korea
2. North_Korea
3. China
4. Taiwan
5. Mongolia
6. Hong_Kong
7. Viet_Nam
8. Thailand
9. Singapore
10. Malaysia

The first 10 areas are:
1. Asia
2. Asia
3. Asia
4. Asia
5. Asia
6. Asia
7. Asia
8. Asia
9. Asia
10. Asia

> Finished chain.
The first 10 country names are:
1. Korea
2. North_Korea
3. China
4. Taiwan
5. Mongolia
6. Hong_Kong
7. Viet_Nam
8. Thailand
9. Singapore
10. Malaysia

The first 10 areas are:
1. Asia
2. Asia
3. Asia
4. Asia
5. Asia
6. Asia
7. Asia
8. Asia
9. Asia
10. Asia


In [22]:
class MyTest:
    def __call__(self):
        self._call()

    def _call(self):
        print('I am called')

# Create an instance
mt = MyTest()

# Call the instance
mt()  # This will print 'I am called'


I am called


In [23]:
agent.dict()

{'name': None,
 'memory': None,
 'verbose': True,
 'tags': None,
 'metadata': None,
 'agent': {'runnable': {'name': None,
   'first': {'name': None,
    'mapper': {'name': None,
     'steps__': {'agent_scratchpad': RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))}}},
   'middle': [{'name': None,
     'input_variables': ['agent_scratchpad', 'input'],
     'optional_variables': [],
     'output_parser': None,
     'partial_variables': {},
     'metadata': None,
     'tags': None,
     'messages': [{'content': '\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nThis is the result of `print(df.head())`:\n|    |   Unnamed: 0 |   Country | Country_name   | Area   |\n|---:|-------------:|----------:|:---------------|:-------|\n|  0 |            0 |       103 | Korea          | Asia   |\n|  1 |            1 |       104 | North_Korea    | Asia   |\n|  2 |            2 |       105 | China          | Asia   |\n|  3 |            3 |       1

- _iter_next_step :  Override this to take control of how the agent makes and acts on choices.Take a single step in the thought-action-observation loop
    - inputs: Dict[str, str] 
    - intermediate_steps: List[Tuple[AgentAction, str]]
    - intermediate_steps = self._prepare_intermediate_steps(intermediate_steps)
    - output = self.agent.plan(intermediate_steps,callbacks=run_manager.get_child() if run_manager else None,
                **inputs, )
    
- self._perform_agent_action-> AgentStep:
    - tool_run_kwargs = self.agent.tool_run_logging_kwargs()
    - if return_direct: tool_run_kwargs["llm_prefix"] = ""
    - observation = tool.run(agent_action.tool_input, **tool_run_kwargs)
    - AgentStep(action=agent_action, observation=observation)
- 


https://github.com/langchain-ai/langchain/blob/02c35da445b5a8cbd3d47ba468243f9297795124/libs/langchain/langchain/agents/react/agent.py#L13
def create_react_agent(
    llm: BaseLanguageModel,
    tools: Sequence[BaseTool],
    prompt: BasePromptTemplate,
    output_parser: Optional[AgentOutputParser] = None,
    tools_renderer: ToolsRenderer = render_text_description,
    *,
    stop_sequence: Union[bool, List[str]] = True,
) -> Runnable:
    """Create an agent that uses ReAct prompting.

    Based on paper "ReAct: Synergizing Reasoning and Acting in Language Models"
    (https://arxiv.org/abs/2210.03629)

    Args:
        llm: LLM to use as the agent.
        tools: Tools this agent has access to.
        prompt: The prompt to use. See Prompt section below for more.
        output_parser: AgentOutputParser for parse the LLM output.
        tools_renderer: This controls how the tools are converted into a string and
            then passed into the LLM. Default is `render_text_description`.
        stop_sequence: bool or list of str.
            If True, adds a stop token of "Observation:" to avoid hallucinates.
            If False, does not add a stop token.
            If a list of str, uses the provided list as the stop tokens.

            Default is True. You may to set this to False if the LLM you are using
            does not support stop sequences.

    Returns:
        A Runnable sequence representing an agent. It takes as input all the same input
        variables as the prompt passed in does. It returns as output either an
        AgentAction or AgentFinish.

    Examples:

        .. code-block:: python

            from langchain import hub
            from langchain_community.llms import OpenAI
            from langchain.agents import AgentExecutor, create_react_agent

            prompt = hub.pull("hwchase17/react")
            model = OpenAI()
            tools = ...

            agent = create_react_agent(model, tools, prompt)
            agent_executor = AgentExecutor(agent=agent, tools=tools)

            agent_executor.invoke({"input": "hi"})

            # Use with chat history
            from langchain_core.messages import AIMessage, HumanMessage
            agent_executor.invoke(
                {
                    "input": "what's my name?",
                    # Notice that chat_history is a string
                    # since this prompt is aimed at LLMs, not chat models
                    "chat_history": "Human: My name is Bob\\nAI: Hello Bob!",
                }
            )

    Prompt:

        The prompt must have input keys:
            * `tools`: contains descriptions and arguments for each tool.
            * `tool_names`: contains all tool names.
            * `agent_scratchpad`: contains previous agent actions and tool outputs as a string.

        Here's an example:

        .. code-block:: python

            from langchain_core.prompts import PromptTemplate

            template = '''Answer the following questions as best you can. You have access to the following tools:

            {tools}

            Use the following format:

            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action
            ... (this Thought/Action/Action Input/Observation can repeat N times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question

            Begin!

            Question: {input}
            Thought:{agent_scratchpad}'''

            prompt = PromptTemplate.from_template(template)
    """  # noqa: E501
    missing_vars = {"tools", "tool_names", "agent_scratchpad"}.difference(
        prompt.input_variables + list(prompt.partial_variables)
    )
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    prompt = prompt.partial(
        tools=tools_renderer(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    if stop_sequence:
        stop = ["\nObservation"] if stop_sequence is True else stop_sequence
        llm_with_stop = llm.bind(stop=stop)  ##### UNDERSTAND THIS IN DETAIL ############################
    else:
        llm_with_stop = llm
    output_parser = output_parser or ReActSingleInputOutputParser()
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | prompt
        | llm_with_stop
        | output_parser
    )
    return agent
    
SINCE LLM IS A RUNNABEL WE UNDERSTAND RUNNNABLE AND THE BIND OPERATION TO RUNNABLE FIRST

In [151]:
# tools_renderer: ToolsRenderer = render_text_description
# https://github.com/langchain-ai/langchain/blob/02c35da445b5a8cbd3d47ba468243f9297795124/libs/core/langchain_core/tools.py#L1352
from inspect import signature
from typing import List
def render_text_description_and_args(tools) -> str:
    """Render the tool name, description, and args in plain text.

    Args:
        tools: The tools to render.

    Returns:
        The rendered text.

    Output will be in the format of:

    .. code-block:: markdown

        search: This tool is used for search, args: {"query": {"type": "string"}}
        calculator: This tool is used for math, \
args: {"expression": {"type": "string"}}
    """
    tool_strings = []
    for tool in tools:
        args_schema = str(tool.args)
        if hasattr(tool, "func") and tool.func:
            sig = signature(tool.func)
            description = f"{tool.name}{sig} - {tool.description}"
        else:
            description = f"{tool.name} - {tool.description}"
        tool_strings.append(f"{description}, args: {args_schema}")
    return "\n".join(tool_strings)

https://github.com/langchain-ai/langchain/blob/02c35da445b5a8cbd3d47ba468243f9297795124/libs/core/langchain_core/tools.py#L1352

In [156]:
from langchain_core.tools import BaseTool
def render_text_description(tools: List[BaseTool]) -> str:
    """Render the tool name and description in plain text.

    Args:
        tools: The tools to render.

    Returns:
        The rendered text.

    Output will be in the format of:

    .. code-block:: markdown

        search: This tool is used for search
        calculator: This tool is used for math
    """
    descriptions = []
    for tool in tools:
        if hasattr(tool, "func") and tool.func:
            sig = signature(tool.func)
            description = f"{tool.name}{sig} - {tool.description}"
        else:
            description = f"{tool.name} - {tool.description}"

        descriptions.append(description)
    return "\n".join(descriptions)

https://github.com/langchain-ai/langchain/blob/02c35da445b5a8cbd3d47ba468243f9297795124/libs/langchain/langchain/agents/format_scratchpad/log.py

In [152]:
from typing import List, Tuple

from langchain_core.agents import AgentAction


def format_log_to_str(
    intermediate_steps: List[Tuple[AgentAction, str]],
    observation_prefix: str = "Observation: ",
    llm_prefix: str = "Thought: ",
) -> str:
    """Construct the scratchpad that lets the agent continue its thought process.

    Args:
        intermediate_steps: List of tuples of AgentAction and observation strings.
        observation_prefix: Prefix to append the observation with.
             Defaults to "Observation: ".
        llm_prefix: Prefix to append the llm call with.
                Defaults to "Thought: ".

    Returns:
        str: The scratchpad.
    """
    thoughts = ""
    for action, observation in intermediate_steps:
        thoughts += action.log
        thoughts += f"\n{observation_prefix}{observation}\n{llm_prefix}"
    return thoughts

- In this case agent is just a chain of:  input_data_dict | passthrough_with_addl_key | promt| model| parser
- The input_data_dict is having a key: intermidiate_steps
- passthrough_with_addl_key: agent_scratchpad whose value is formatted thought and observation intermidiate_steps
- this dict goes to prompt to fill up the variables in the prmpt
- prompt goes to model
- output of the model is parsed
- we need to find who is calling this agent.invoke with the assumed input_data_dict

In [163]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import BasePromptTemplate
from langchain.agents import AgentOutputParser
from typing import List, Optional, Sequence, Union
from langchain.tools.render import ToolsRenderer, render_text_description

def create_react_agent(
    llm: BaseLanguageModel,
    tools: Sequence[BaseTool],
    prompt: BasePromptTemplate,
    output_parser: Optional[AgentOutputParser] = None,
    tools_renderer: ToolsRenderer = render_text_description,
    *,
    stop_sequence: Union[bool, List[str]] = True,
) -> Runnable:
    missing_vars = {"tools", "tool_names", "agent_scratchpad"}.difference(
            prompt.input_variables + list(prompt.partial_variables)
        )
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    prompt = prompt.partial(
        tools=tools_renderer(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    if stop_sequence:
        stop = ["\nObservation"] if stop_sequence is True else stop_sequence
        llm_with_stop = llm.bind(stop=stop)
    else:
        llm_with_stop = llm
    output_parser = output_parser or ReActSingleInputOutputParser()
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | prompt
        | llm_with_stop
        | output_parser
    )
    return agent



In [168]:
import sys
sys.path.append("llmapslab")
import os
import json
import re
import load_configs
import importlib
importlib.reload(load_configs)
from load_configs import (
    openai_api_key,
    llama_api_key, ai21_api_key, 
    gemini_api_key
)
from openai_client import call_openai_api

In [173]:
# !pip install langchainhub
from langchain import hub
from langchain_community.llms import OpenAI
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
model = OpenAI(openai_api_key=openai_api_key)
tools = []

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, max_iterations=1)

agent_executor.invoke({"input": "hi"})

# # Use with chat history
# from langchain_core.messages import AIMessage, HumanMessage
# agent_executor.invoke(
#     {
#         "input": "what's my name?",
#         # Notice that chat_history is a string
#         # since this prompt is aimed at LLMs, not chat models
#         "chat_history": "Human: My name is Bob\\nAI: Hello Bob!",
#     }
# )

/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "hi"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": ""
}
[chain/end] [chain:Age

/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2

[llm/end] [chain:AgentExecutor > chain:RunnableSequence > llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " You should always greet and acknowledge someone when they say hi\nAction: [print]\nAction Input: \"Hello!\"",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > parser:ReActSingleInputOutputParser] Entering Parser run with input:
{
  "input": " You should always greet and acknowledge someone when they say hi\nAction: [print]\nAction Input: \"Hello!\""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > parser:ReActSingleInputOutputParser] s] Exiting Parser run with output:
[outputs]
[chain/end] [chain:AgentExecutor > chain:RunnableSequence] s] Exiting Chain run with output:
[outputs]
[tool/start] [chain:AgentExecutor > tool:inva

/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2

{'input': 'hi',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [171]:
help(AgentExecutor)

Help on class AgentExecutor in module langchain.agents.agent:

class AgentExecutor(langchain.chains.base.Chain)
 |  AgentExecutor(*, name: Optional[str] = None, memory: Optional[langchain_core.memory.BaseMemory] = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, verbose: bool = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, agent: Union[langchain.agents.agent.BaseSingleActionAgent, langchain.agents.agent.BaseMultiActionAgent], tools: Sequence[langchain_core.tools.BaseTool], return_intermediate_steps: bool = False, max_iterations: Optional[int] = 15, max_execution_time: Optional[float] = None, early_stopping_method: str = 'force', handle_parsing_errors: Union[bool, str, Callable[[langchain_core.exceptions.OutputParserException], str]] = False, trim_intermediate_steps: 